In [1]:
from dotenv import load_dotenv


from langchain_groq import ChatGroq


from langgraph.graph import StateGraph , START , END
from langgraph.prebuilt import ToolNode , tools_condition

from pydantic import BaseModel , Field

from typing_extensions import TypedDict , Literal
from langgraph.graph.message import add_messages , Annotated 
from langchain.messages import HumanMessage , AIMessage , AnyMessage , SystemMessage

import os

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Router(BaseModel) :

    topic : Literal["joke" , "horror"] = Field(description= "Sentence subject")

In [3]:
load_dotenv()

True

In [4]:
llm = ChatGroq(model = "openai/gpt-oss-20b")

In [5]:
router = llm.with_structured_output(Router)

In [6]:
class State(TypedDict) :
    input : str 
    decision : str
    output : str

In [20]:
def call_1(state : State) :
    """Write a joke"""

    response = llm.invoke(state["input"])

    return {"output" : response.content}

In [21]:
def call_2(state : State) :
    """Write a horror sentence"""

    response = llm.invoke(state["input"])

    return {"output" : response.content}

In [30]:
def llm_call_router(state : State) :
    """Route the input to the appropriate node"""

    decision = router.invoke( 
        [
            SystemMessage(content= "Route the input to joke or horror based on user request.") , 
            HumanMessage(content= state["input"])
        ]

    )

    return {"decision" : decision.topic}

In [31]:
def route_decision(state : State) :

    if state["decision"] == "joke" :
        return "joke"
    
    elif state["decision"] == "horror" :
        return "horror"
    

In [32]:
graph = StateGraph(State)

In [33]:
graph.add_node("joke" , call_1)
graph.add_node("horror", call_2)
graph.add_node("llm_call_router" , llm_call_router)

In [34]:
graph.add_edge(START , "llm_call_router")

graph.add_conditional_edges("llm_call_router" , route_decision , {"joke" : "joke" , "horror" : "horror" })

graph.add_edge("joke" , END)
graph.add_edge("horror" , END)

graph_builder =graph.compile()

In [35]:
state = graph_builder.invoke({"input" : "Tell me a joke about agentic AI"})

In [36]:
state

{'input': 'Tell me a joke about agentic AI',
 'decision': 'joke',
 'output': 'Why did the agentic AI bring a ladder to the meeting?\n\nBecause it heard the company was “climbing the ladder of success” and wanted to *actually* take the initiative!'}

In [12]:
#router.invoke("I tried to catch fog yesterday, but I mist.")